# Algebra Linear Algorítmica - Trabalho Final
##  Alunos
- Antonio Pedro
- Diogo Lima

## Descrição
Neste projeto desenvolveremos um software que recebe alimentos e o objetivo  alimentar de um individuo e retorna a dieta ideal do mesmo.

### Dados de entrada:
- Peso [kg]
- Idade [ano]
- Gênero [M/F]
- Lista de preferencia alimentar

In [55]:
PESO = 70.0
IDADE = 24
GENERO = "M"

LISTA_ALIMENTOS = [2, 3, 27, 123, 9]

5-element Vector{Int64}:
   2
   3
  27
 123
   9

## Bibliotecas Utilizadas

In [56]:
using LinearAlgebra
using CSV
using DataFrames

### Objetivos alimentares
A quantidade ideal de cada nutriente ingerido por dia:
- **Calorias**:
    - Homens: 2400 [kcal]/dia
    - Mulheres: 2000 [kcal]/dia
    - fonte: https://oglobo.globo.com/saude/noticia/2023/01/quantas-calorias-devo-consumir-por-dia.ghtml

- **Proteína**: 
    - Sem exercício: 1.0 [g/kg]/dia
    - Com exercício: 2.0 [g/kg]/dia
    - Consideraremos 1.5 [g/kg]/dia
    - fonte: https://www.metropoles.com/saude/como-calcular-quantidade-proteina-por-dia

- **Lipídeos (Gordura)**
    - 30% da Caloria diária (ou menos)
    - 1g tem cerca de 9 kcal
    - fonte caloria diária: https://www.who.int/news/item/17-07-2023-who-updates-guidelines-on-fats-and-carbohydrates
    - fonte conversão g -> kcal: https://brasilescola.uol.com.br/curiosidades/contando-calorias.htm#:~:text=Um%20grama%20de%20carboidrato%20oferece,fornece%20cerca%20de%209%20kcal.

- **Colesterol**
    - 10% da caloria diária (ou menos)
    - 22g tem cerca de 200 kcal (9 kcal/g)
    - fonte: https://www.healthline.com/health/high-cholesterol/rda

- **Carboidrato**
    - 4 a 6 [g/kg]/dia
    - Consideraremos 5 [g/kg]/dia
    - fonte: https://www.terra.com.br/vida-e-estilo/quantas-gramas-diarias-de-carboidrato-sao-necessarias-para-hipertrofia-entenda,f5f7ce9ed991948b24cbd30bc1e8f9c6njbq608z.html#:~:text=%22Um%20consumo%20m%C3%A9dio%20de%20carboidratos,g%20de%20carboidratos%20por%20dia.

- **Fibra Alimentar**
    - 2-5 anos: 15 g/dia
    - 6-9 anos: 21 g/dia
    - 10+ anos: 25 g/dia
    - fonte: https://www.who.int/news/item/17-07-2023-who-updates-guidelines-on-fats-and-carbohydrates

- **Cálcio**
    - < 1 ano: 260 mg/dia
    - 1-3 anos: 700 mg/dia
    - 4-8 anos: 1000 mg/dia
    - 9-18 anos: 1300 mg/dia
    - 18-70 anos: 1000 mg/dia
    - 70+ anos: 1200 mg/dia
    - fonte: https://www.unimed.coop.br/viver-bem/alimentacao/qual-a-quantidade-de-calcio-que-o-corpo-precisa-e-como-obte-la

- **Magnésio**
    - < 1 ano: 30 mg/dia
    - 1-3 anos: 75 mg/dia
    - 4-8 anos: 80 mg/dia
    - 9-13 anos: 240 mg/dia
    - 14-18 anos: 
        - homem: 410 mg/dia
        - mulher: 360 mg/dia
    - 19-30 anos: 
        - homem: 400 mg/dia
        - mulher: 310 mg/dia
    - 31+ anos: 
        - homem: 420 mg/dia
        - mulher: 320 mg/dia
    - fonte: https://www.healthline.com/nutrition/magnesium-dosage#supplements


In [57]:
function get_dosagens_diarias(idade, peso, genero)
    """Consideraremos a idade em anos, peso em Kg e genero "M" ou "F" """
    vetor_ideal = zeros(8)
    
    # Defendendo Entradas
    if (genero != "M" && genero != "F") || (idade < 0) || (peso < 0)
        return false, vetor_ideal
    end

    # Calorias
    if genero == "M"
        vetor_ideal[1] = 2400
    else
        vetor_ideal[1] = 2000
    end

    # Proteinas
    vetor_ideal[2] = peso * 1.5

    # Lipídeos
    vetor_ideal[3] = (vetor_ideal[1] * 0.3) / 9

    # Colesterol
    vetor_ideal[4] = (vetor_ideal[1] * 0.1) / 9

    # Carboidratos
    vetor_ideal[5] = peso * 5

    # Fibra Alimentar
    if (idade < 2)
        vetor_ideal[6] = 0
    
    elseif (idade < 5)
        vetor_ideal[6] = 15
        
    elseif (idade < 10)
        vetor_ideal[6] = 21
        
    else
        vetor_ideal[6] = 25
    end

    # Calcio
    if (idade < 1)
        vetor_ideal[7] = 260
    
    elseif (idade < 3)
        vetor_ideal[7] = 300

    elseif (idade < 8)
        vetor_ideal[7] = 1000

    elseif (idade < 18)
        vetor_ideal[7] = 1300

    elseif (idade < 70)
        vetor_ideal[7] = 1000

    else
        vetor_ideal[7] = 1200
    end
    
    # Magnésio
    if (idade < 1)
        vetor_ideal[8] = 30

    elseif (idade < 3)
        vetor_ideal[8] = 75

    elseif (idade < 8)
        vetor_ideal[8] = 80

    elseif (idade < 13)
        vetor_ideal[8] = 240

    elseif (idade < 18)
        if (genero == "M")
            vetor_ideal[8] = 410
        else
            vetor_ideal[8] = 360
        end
        
    elseif (idade < 30)
        if (genero == "M")
            vetor_ideal[8] = 400
        else
            vetor_ideal[8] = 310
        end

    else
        if (genero == "M")
            vetor_ideal[8] = 420
        else
            vetor_ideal[8] = 320
        end
    end

    return true, vetor_ideal
end

validade, dosagem_diaria = get_dosagens_diarias(IDADE, PESO, GENERO)
if !validade
    print("Parametro invalido")
end

print(dosagem_diaria)

[2400.0, 105.0, 80.0, 26.666666666666668, 350.0, 25.0, 1000.0, 400.0]

## Dados Utilizados

### Base de alimentos
Os dados refletem as quantidades de nutriente para mais de 300 alimentos.

Fonte de dados: https://www.kaggle.com/datasets/ispangler/composio-nutricional-de-alimentos-taco?select=taco-db-nutrientes.csv

In [58]:
df = CSV.read("arquivo_sem_cru.csv", DataFrame)
display(df)

Row,Nome,Energia (kcal),Proteína (g),Lipídeos (g),Colesterol (mg),Carboidrato (g),Fibra Alimentar (g),Cálcio (mg),Magnésio (mg)
,String,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Int64
1,Arroz integral cozido,124.0,2.6,1.0,0,25.8,2.7,5.0,59
2,Arroz cozido,128.0,2.5,0.2,0,28.1,1.6,4.0,2
3,Biscoito doce maisena,443.0,8.1,12.0,0,75.2,2.1,54.0,37
4,Biscoito doce recheado com chocolate,472.0,6.4,19.6,0,70.5,3.0,27.0,48
5,Biscoito doce recheado com morango,471.0,5.7,19.6,0,71.0,1.5,36.0,27
6,Biscoito doce wafer recheado de chocolate,502.0,5.6,24.7,0,67.5,1.8,23.0,48
7,Biscoito doce wafer recheado de morango,513.0,4.5,26.4,1,67.4,0.8,14.0,19
8,Biscoito salgado cream cracker,432.0,10.1,14.4,0,68.7,2.5,20.0,40
9,Bolo mistura para,419.0,6.2,6.1,0,84.7,1.7,59.0,28


### Tratando dados
Convertemos as colunas para o tipo correto de dados

In [59]:
for nome in names(df)[3:ncol(df)]
    print(nome, ", ")
    convert(Vector{Float64}, df[!, nome])
end

display(DataFrames.first(df, 5))

Row,Nome,Energia (kcal),Proteína (g),Lipídeos (g),Colesterol (mg),Carboidrato (g),Fibra Alimentar (g),Cálcio (mg),Magnésio (mg)
,String,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Int64
1,Arroz integral cozido,124.0,2.6,1.0,0,25.8,2.7,5.0,59
2,Arroz cozido,128.0,2.5,0.2,0,28.1,1.6,4.0,2
3,Biscoito doce maisena,443.0,8.1,12.0,0,75.2,2.1,54.0,37
4,Biscoito doce recheado com chocolate,472.0,6.4,19.6,0,70.5,3.0,27.0,48
5,Biscoito doce recheado com morango,471.0,5.7,19.6,0,71.0,1.5,36.0,27


Proteína (g), Lipídeos (g), Colesterol (mg), Carboidrato (g), Fibra Alimentar (g), Cálcio (mg), Magnésio (mg), 

### Escalonando Dados
Escalonaremos os dados em função da quantidade de nutrientes demandados por cada indivíduo, dividindo a quantidade presente na tabela pelo valor alvo para atingirmos um percentual do nutriente alvo e não mais um valor absoluto

In [93]:
for i in 2: (ncol(df))
    df[!, i] .= (df[!, i] / dosagem_diaria[i - 1]) * 100
end

vetor_alvo = ones(length(dosagem_diaria)) * 100

for i in 2: ncol(df)
    df[!, i] .= round.(df[!, i], digits=2)
end

display(first(df, 5))
display(vetor_alvo)

Row,Nome,Energia (kcal),Proteína (g),Lipídeos (g),Colesterol (mg),Carboidrato (g),Fibra Alimentar (g),Cálcio (mg),Magnésio (mg)
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Arroz integral cozido,0.22,2.36,1.56,0.0,2.11,43.2,0.05,3.69
2,Arroz cozido,0.22,2.27,0.31,0.0,2.29,25.6,0.04,0.12
3,Biscoito doce maisena,0.77,7.34,18.75,0.0,6.14,33.6,0.54,2.31
4,Biscoito doce recheado com chocolate,0.82,5.81,30.63,0.0,5.75,48.0,0.27,3.0
5,Biscoito doce recheado com morango,0.82,5.17,30.63,0.0,5.8,24.0,0.36,1.69


8-element Vector{Float64}:
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0

## Criando Matriz Base
Com base nos alimentos selecionados, temos a seguinte matriz $A$

In [61]:
if (length(LISTA_ALIMENTOS) == 0)
    print("Nenhum alimento selecionado! Por favor adicione indices de alimentos na variavel LISTA_ALIMENTOS")

else
    A = Vector(df[LISTA_ALIMENTOS[1]-1, 2:ncol(df)])

    for idx in LISTA_ALIMENTOS[2: length(LISTA_ALIMENTOS)]
        A = hcat(A, Vector(df[idx - 1, 2:ncol(df)]))
    end

    display(A)
end

vetorAlimentos = A

8×5 Matrix{Float64}:
  5.17  5.33  18.17  24.71  18.0
  2.48  2.38   8.38   0.0    9.62
  1.25  0.25  21.5   83.87  18.0
  0.0   0.0    0.0    0.0    0.0
  7.37  8.03  17.83   0.0   19.63
 10.8   6.4   22.4    0.0   10.0
  0.5   0.4    1.8    0.5    2.0
 14.75  0.5    4.75   0.25  10.0

8×5 Matrix{Float64}:
  5.17  5.33  18.17  24.71  18.0
  2.48  2.38   8.38   0.0    9.62
  1.25  0.25  21.5   83.87  18.0
  0.0   0.0    0.0    0.0    0.0
  7.37  8.03  17.83   0.0   19.63
 10.8   6.4   22.4    0.0   10.0
  0.5   0.4    1.8    0.5    2.0
 14.75  0.5    4.75   0.25  10.0

## Calcula Projeção
Recebe o vetor Alvo (que contém os dados  referentes à alimentação balanceada) e o vetor que contém os dados dos alimentos escolhidos pelo usuário. Dessa forma, iremos encontrar a projeção do vetor ideal de consumo no subespaço gerado pelos vetores dos alimentos. Essa projeção é a combinação linear do subespaço desses vetores mais próxima possível do vetor ideal de consumo.

In [62]:
function projecao(vetorAlvo, vetorGeral)

  A = vetorGeral

  matrizProjecao = A * inv((A' * A)) * A' 

  display(matrizProjecao)

  projecao = matrizProjecao*vetorAlvo

  projecao = round.(projecao, digits = 2)

  display(projecao)
  return projecao
end

vetorProjecao = projecao(vetor_alvo,vetorAlimentos);



8×8 Matrix{Float64}:
  0.329736     0.00837712    0.197537    0.0  …  -0.00744319  -0.00583758
  0.00837712   0.944922     -0.00382547  0.0      0.227944    -0.000515145
  0.197537    -0.00382547    0.94175     0.0      0.00781638   0.00170591
  0.0          0.0           0.0         0.0      0.0          0.0
  0.4262       0.000381032  -0.125467    0.0     -0.0189243    0.00377299
  0.0135466   -0.0010292    -0.0040136   0.0  …   0.00371443   0.000108783
 -0.00744319   0.227944      0.00781638  0.0      0.0555369    0.00237271
 -0.00583758  -0.000515145   0.00170591  0.0      0.00237271   0.999943

8-element Vector{Float64}:
  96.21
 117.63
 101.55
   0.0
 100.58
 100.35
  27.1
 100.16

## Calcula Quantidade de cada Alimento

Após encontrarmos a projeção, realizamos a resolução do sistema linear Ax = b, em que A é  vetor com as dados dos alimentos, e b é a projeção. Já que a projeção está no subespaço gerado pelos vetores dos alimentos, é garantido que haverá uma solução para o sistema.

In [98]:
function calculaAlimentoIndices(vetorGeral, vetorProjecao)     
    x = vetorGeral \ vetorProjecao
    x = round.(x, digits= 2)
    return x
end 

indicesAlimentos = calculaAlimentoIndices(vetorAlimentos, vetorProjecao)


## Transformamos as quantidades encontradas para gramas

function calculaAlimentosGramas(indicesAlimentos)
    for i in eachindex(indicesAlimentos)
        indicesAlimentos[i] *= 100
    end
end

calculaAlimentosGramas(indicesAlimentos)

print(indicesAlimentos)



[-440.00000000000006, -4387.0, 1362.0, -474.0, 1235.0]

In [ ]:
function printaAlimentos(indicesAlimentos)
    

end 
  